In [6]:
import sys
sys.setrecursionlimit(10**6)
import pickle
import os
import json
import numpy as np
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from cleanlab.classification import LearningWithNoisyLabels
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from dataload import *
import operator

In [2]:
def get_best_params(job, results): 
    _, d, e, s, ex = job
    split_seed = s.split("_")[1]
    ex_seed = ex[:-2].split("_")[1]
    key = "{}/v{}/{}/dirty/{}/{}".format(d, split_seed, e, model_name, ex_seed)
    
    max_acc = 0
    a =  results[key].copy()
    #print(a)
    del a['best_params']
    del a['train_acc']
    del a['val_acc']
    max_key = max(a.items(), key=operator.itemgetter(1))[0]

    if a[max_key] > max_acc:
        best_params = results[key]['best_params']
        max_acc = a[max_key]
    
    print('best_params:', best_params)
    return best_params

In [3]:
data_dir = "data-robustml"
save_dir = "result-robustml"

#model_name = sys.argv[1] # "logistic_regression"

model_name = "adaboost_classification"

In [ ]:
jobs = get_jobs(data_dir, error_type = 'mislabel', dataset = ['EEG_major', 'EEG_minor', 'EEG_uniform', 'Marketing_major', 'Marketing_minor', 'Marketing_uniform', 'Titanic_uniform', 'Titanic_major', 'Titanic_minor', 'USCensus_uniform', 'USCensus_major', 'USCensus_minor'])

print(jobs)
print("Number of jobs:", len(jobs))

running clothing

In [4]:
jobs = get_jobs(data_dir, error_type = 'mislabel', dataset = ['Clothing'])
print(jobs)
print("Number of jobs:", len(jobs))

[['data-robustml', 'Clothing', 'mislabel', 'split_7813', 'data_3659.p'], ['data-robustml', 'Clothing', 'mislabel', 'split_2516', 'data_3659.p'], ['data-robustml', 'Clothing', 'mislabel', 'split_3462', 'data_3659.p'], ['data-robustml', 'Clothing', 'mislabel', 'split_3562', 'data_3659.p'], ['data-robustml', 'Clothing', 'mislabel', 'split_235', 'data_3659.p'], ['data-robustml', 'Clothing', 'mislabel', 'split_5056', 'data_3659.p'], ['data-robustml', 'Clothing', 'mislabel', 'split_4225', 'data_3659.p'], ['data-robustml', 'Clothing', 'mislabel', 'split_5374', 'data_3659.p'], ['data-robustml', 'Clothing', 'mislabel', 'split_9394', 'data_3659.p'], ['data-robustml', 'Clothing', 'mislabel', 'split_6542', 'data_3659.p'], ['data-robustml', 'Clothing', 'mislabel', 'split_144', 'data_3659.p'], ['data-robustml', 'Clothing', 'mislabel', 'split_905', 'data_3659.p'], ['data-robustml', 'Clothing', 'mislabel', 'split_8444', 'data_3659.p'], ['data-robustml', 'Clothing', 'mislabel', 'split_4764', 'data_3659

In [12]:
for job in jobs:
    
    print("Processing", job)
    dataset = job[1]
    
    data = load_data(job)
    
    with open('/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/CleanML/result/{}_result.json'.format(dataset)) as json_file:
        results = json.load(json_file)
    best_params = get_best_params(job, results) 
    
    X_train, y_train = data["train"]

    result_dict = {}
    ### TODO: train model (tune hyperparameters)
    lnl = LearningWithNoisyLabels(clf=AdaBoostClassifier(n_estimators = 200, learning_rate = best_params['learning_rate']))
    lnl.fit(X=X_train, s=y_train)
    
    # optinal: save hyperparams
    result_dict["best_params"] = best_params
    ### END

    for version, (X_test, y_test) in data["test"].items():
        ### TODO: prediction
        y_pred = lnl.predict(X_test)
        ### END

        test_f1 = f1_score(y_test, y_pred, average='macro')
        test_acc = np.mean(y_pred == y_test)

        result_dict["{}_test_f1".format(version)] = test_f1
        result_dict["{}_test_acc".format(version)] = test_acc

    print(result_dict)
    save_result(job, result_dict, model_name, save_dir)

Processing ['data-robustml', 'Clothing', 'mislabel', 'split_7813', 'data_3659.p']
best_params: {'learning_rate': 0.11775669666586115}
{'best_params': {'learning_rate': 0.11775669666586115}, 'dirty_test_f1': 0.2056520605837051, 'dirty_test_acc': 0.262, 'clean_test_f1': 0.19702244822659867, 'clean_test_acc': 0.23266666666666666}
Processing ['data-robustml', 'Clothing', 'mislabel', 'split_2516', 'data_3659.p']
best_params: {'learning_rate': 0.11775669666586115}
{'best_params': {'learning_rate': 0.11775669666586115}, 'dirty_test_f1': 0.1850162167954715, 'dirty_test_acc': 0.234, 'clean_test_f1': 0.18867534236801595, 'clean_test_acc': 0.214}
Processing ['data-robustml', 'Clothing', 'mislabel', 'split_3462', 'data_3659.p']
best_params: {'learning_rate': 0.11775669666586115}
{'best_params': {'learning_rate': 0.11775669666586115}, 'dirty_test_f1': 0.20810906456730174, 'dirty_test_acc': 0.26666666666666666, 'clean_test_f1': 0.19250012883845735, 'clean_test_acc': 0.228}
Processing ['data-robustml